# Lab 4.1 : Spark Dataframes


### Overview
First look at Spark Dataframes

### Depends On 
None

### Run time
20-30 mins


In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()

## STEP 1: Load Clickstream data

The clickstream data looks like this

```json
{"timestamp": 1420070400000, "ip": "ip_557", "user": "user_13011", "action": "blocked", "domain": "npr.org", "campaign": "campaign_13", "cost": 116, "session": "session_43"}

{"timestamp": 1420070400043, "ip": "ip_129", "user": "user_58773", "action": "clicked", "domain": "flickr.com", "campaign": "campaign_7", "cost": 170, "session": "session_23"}

{"timestamp": 1420070400086, "ip": "ip_704", "user": "user_71191", "action": "viewed", "domain": "foxnews.com", "campaign": "campaign_20", "cost": 47, "session": "session_48"}
```

In [ ]:
clickstreamDF = spark.read.json("../data/click-stream/clickstream.json")
print(clickstreamDF)

**==> Monitor Spark shell UI on port 4040+**  
You may see something like this:

**==> Q : Why is Spark not lazy loading the JSON files?**

<img src="../assets/images/5.1a.png" style="border: 5px solid grey; max-width:100%;" />




## STEP 2 : Inspecting The Dataframe

**==> Print the schema of data frame**     


In [ ]:
clickstreamDF.printSchema()

**==> Print / Dump the data contained within dataframe**  
Your output may look like this:

```
+-------+-----------+----+-----------------+----+----------+-------------+------+
| action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
+-------+-----------+----+-----------------+----+----------+-------------+------+
|clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
|blocked|campaign_12|   5|     facebook.com|ip_3|session_96|1420070400864|user_5|
|clicked| campaign_3|  54|sf.craigslist.org|ip_9|session_61|1420070401728|user_8|
...

```

**==> Let's see the data**

In [ ]:
clickstreamDF.show()

**==> Explore methods available in Dataframe**  
Here is the Dataframe API : 
[Scala](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrame)  /
[Java](http://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrame.html) / 
[Python](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe#pyspark.sql.DataFrame)



## STEP 3 : Querying Dataframe

**==> Show only click logs where the cost > 100**

Sample output

```
    +-------+-----------+----+-----------------+----+----------+-------------+------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|
    +-------+-----------+----+-----------------+----+----------+-------------+------+
    |clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|
    |blocked|campaign_18| 110|    wikipedia.org|ip_5|session_55|1420070402592|user_6|
    |blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|
```

In [ ]:
clickstreamDF.filter("cost > ???").show()

In [ ]:
#Same thing
clickstreamDF.filter(clickstreamDF['cost'] > ???).show()

**==> Show the logs where action = clicked**  

In [ ]:
# TODO: show the logs where action = clicked
# Hint : clickstreamDF.filter("action == '???'")
clickstreamDF.filter ("??? == '???'")

In [ ]:
## TODO Another approach
clickstreamDF.filter(clickstreamDF['column_name'] == '???').show()

### Quick Hack on Displaying Dataframes in Pretty Format

Spark `df.show()` will print in plain text.  It can be difficult to look at, for dataframes with large number of columns.   Pandas can help here.

We can easily convert Spark dataframes to Pandas dataframes.

Few things to note:
* When converting Spark --> Pandas, be mindful of size.  You don't want to accidentally convert huge Spark dataframe into Pandas.  Always use **limit**

In [ ]:
import pandas as pd

clickstreamDF.filter ("action == 'clicked'").limit(10).toPandas()

## Step 4 : Count the number of visits from each domain

Hint : `clickstreamDF.groupBy("domain").count().show()`  


In [ ]:
## TODO: 
## Hint: groupBy ("domain")
clickstreamDF.groupBy("???").count()

In [ ]:
clickstreamDF.groupBy("???").count().show()

### Inspect the Spark application UI (4040+)

Here you will optimizer kicking in!

* Look at number of tasks (sometimes can be 100!) - but only a few tasks will actually get data.  Verify this
* some stages will be skipped.  Because the optimizer can reuse previous results!
* Inspect the DAG for the job.  You will see skipped stages

![](../assets/images/dataframe-1.png)

![](../assets/images/dataframe-2.png)

## STEP 5 : Joining Dataframes

Let's load another data set `domain info`  
The data is in   `/data/click-stream/domain-info.json`  
The data looks like this:

```json
    {"domain":"amazon.com","category":"SHOPPING"}
    {"domain":"bbc.co.uk","category":"NEWS"}
    {"domain":"facebook.com","category":"SOCIAL"}
    ...
```

**==> Load the dataframe**

In [ ]:
domainsDF = spark.read.json("../data/click-stream/domain-info.json")
print(domainsDF)


In [ ]:
## TODO display domains data
## Hint : show()
domainsDF.???()

**==> Join both dataframes**

sample output
```
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|   category|           domain|
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    |clicked|campaign_19| 118|      youtube.com|ip_4|session_36|1420070400000|user_9|      VIDEO|      youtube.com|
    |blocked| campaign_2|   7|      youtube.com|ip_2|session_93|1420070412960|user_1|      VIDEO|      youtube.com|
    |blocked|campaign_17|  20|       amazon.com|ip_4|session_13|1420070406048|user_1|   SHOPPING|       amazon.com|
```

In [ ]:

joined = clickstreamDF.join(domainsDF,  clickstreamDF["domain"] == domainsDF["domain"])
    
#see the results
joined.show()

**==> Note some rows are missing.  Which ones?  Why?**

### Inspect Optimizations!

open SQL tab and select the query and inspect the DAG.

You will see optimizer doing some work for us already.

* It is filtering on `domain != null` .  Why is that?
* Also notice it is joing `broadcast` join for us

![](../assets/images/dataframe-3.png)

![](../assets/images/dataframe-4.png)

![](../assets/images/dataframe-5.png)

## Step-6: Do an OUTTER JOIN

Inspect the output, might look like this

**==> Can you explain the null values?**

Output:

```
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    | action|   campaign|cost|           domain|  ip|   session|    timestamp|  user|   category|           domain|
    +-------+-----------+----+-----------------+----+----------+-------------+------+-----------+-----------------+
    |blocked| campaign_9| 139|          cnn.com|ip_8|session_13|1420070404320|user_7|       null|             null|
    |   null|       null|null|             null|null|      null|         null|  null|     SOCIAL|      twitter.com|
    |clicked| campaign_6|  15|comedycentral.com|ip_9|session_49|1420070403456|user_4|       null|             null|
```

In [ ]:

joinedOuter = clickstreamDF.join(domainsDF,  clickstreamDF["domain"] == domainsDF["domain"], "outer")
joinedOuter.show()

## Step 7: Understanding Query Execution

We will use **explain** keyword

In [ ]:
clickstreamDF.filter("cost > 100").explain(extended=True)

In [ ]:
joined.explain(extended=True)

In [ ]:
joinedOuter.explain(extended=True)

### Notice the optimizations!

There are few optimizations here

* broadcast join
* filtering on `domain != null`
* pushed down predicates  `domain != null`

![](../assets/images/dataframe-6-explain.png)